# Create auxiliary data for CONUS maps
- Includes DEM, location and climate zone data
- DEM and location is from GEE. Climate zone from a local GeoTiff
- Includes water mask
- Converts to EPSG:4326, scale 10km
- Runs in 3 parts. The first two parts submit GEE tasks, so need to wait for these to complete before running the next part

## Part 1: Create images in Google Earth Engine

In [1]:
import numpy as np
import os
import initialise
import common
import ee
ee.Initialize()

In [2]:
# Required projection and resolution
PROJ = common.MAP_PROJ
SCALE = common.MAP_SCALE
BBOX_PROJ = "EPSG:4326"

# Location and name of aux image
FOLDER = f"GEE_{PROJ.replace(':', '-')}_{int(SCALE)}"
GEE_DATA_DIR = os.path.join(common.GEE_MAPS_DIR, FOLDER)
GEE_AUX_IMAGE = 'conus_aux_gee.tif'
FINAL_AUX_IMAGE = 'conus_aux.tif'
if not os.path.exists(GEE_DATA_DIR):
    os.makedirs(GEE_DATA_DIR)

# DEM product name
DEM_PRODUCT = 'USGS/SRTMGL1_003'

# Water mask product and details
WATER_MASK_PRODUCT = 'MODIS/006/MOD44W'
WATER_MASK_BAND = 'water_mask'
WATER_MASK_START_DATE = '2015-01-01'
WATER_MASK_END_DATE = '2015-05-01'

# USA shape and CONUS bounding box for clipping images
USA = ee.FeatureCollection('USDOS/LSIB/2017').filter("COUNTRY_NA == 'United States'")
long_lat_rect = [-125, 24, -65, 50]
#conus_bbox = ee.Geometry.Rectangle(long_lat_rect, BBOX_PROJ, geodesic=False)
conus_bbox = ee.Geometry.BBox(*long_lat_rect)

# GEE assest location for auxiliary images
GEE_FOLDER = 'users/xxxxx' # Replace xxxxx with your GEE username
GEE_ASSET = f'{GEE_FOLDER}/Auxiliary2'

KOPPEN_FILE = 'Beck_KG_V1_present_0p0083.tif'
NODATA = common.GDAL_NODATA_VALUE

DEM image processing

In [3]:
dem_image = ee.Terrain.products(ee.Image(DEM_PRODUCT)).reduceResolution(ee.Reducer.mean(), maxPixels=512)
dem_image = dem_image.select(['elevation', 'slope', 'aspect'])
if common.MAP_SCALE > 500:
    temp_scale = common.MAP_SCALE // int((common.MAP_SCALE / 30) ** 0.5)
    dem_image = dem_image.reproject(common.MAP_PROJ, scale=temp_scale).reduceResolution(ee.Reducer.mean(), maxPixels=512)

Create an image for the longitude and latitude

In [4]:
long_lat = ee.Image.pixelLonLat()

Get the water mask

In [5]:
water_mask = ee.ImageCollection(WATER_MASK_PRODUCT).filter(ee.Filter.date(WATER_MASK_START_DATE, WATER_MASK_END_DATE))
water_mask = water_mask.select(WATER_MASK_BAND).toBands()

Combine the images and water mask

In [6]:
full_image = dem_image.addBands(long_lat).updateMask(ee.Image.constant(1).subtract(water_mask))

### Store image as GEE Asset
GEE limitations on the Terrain product mean there's too much data to generate as a single image. Seems like about 10 (square) degrees is about the limit. So need to create a set of small images, then mosaic into a single image. So first create an image collection (if it doesn't already exist), then iterate through the longitudes and latitudes to create images and submit a task to create the image as a GEE asset. At 5x2 degrees, 112 images & tasks are generated.  
Note: When the images are mosaicked and displayed in GEE, there are distinct bands where the images meet. If the mosaicked image is exported as a GeoTiff, the bands are not visible (e.g. when displayed in QGIS).

In [ ]:
try:
    ee.data.createAsset({'type': 'ImageCollection'}, GEE_ASSET)
except:
    raise Exception('Image collection already exists')

In [ ]:
long_size = 5
lat_size = 2
long_range = range(long_lat_rect[0], long_lat_rect[2], long_size)
lat_range = range(long_lat_rect[1], long_lat_rect[3], lat_size)
for long in long_range:
    for lat in lat_range:
#        bbox = ee.Geometry.Rectangle([long, lat, long+long_size, lat+lat_size], BBOX_PROJ, geodesic=False)
        bbox = ee.Geometry.Rectangle(long, lat, long+long_size, lat+lat_size)
        if bbox.intersects(USA.geometry(), 1).getInfo():
            print(f'Aux_{lat:02}{long:04}', f'{GEE_ASSET}/{lat:02}{long:04}')
            bbox = bbox.intersection(USA.geometry(), 1, BBOX_PROJ)
            image = full_image.clip(bbox)
            task = ee.batch.Export.image.toAsset(
                image=image,
                description=f'Aux_{lat:02}{long:04}',
                assetId=f'{GEE_ASSET}/{lat:02}{long:04}',
                scale=SCALE,
                crs=PROJ,
            );
            task.start()

## Part 2: Export mosaicked image
Creates a mosaicked image from the image collection created previously, then exports to Google Drive as a GeoTiff.  
Notes:
1. Need to wait for the GEE tasks generated above to complete before running this cell.
2. Run the first two cells in the notebook before running this one if the kernel has been restarted since running part 1

In [9]:
conus_aux = ee.ImageCollection(GEE_ASSET).mosaic().toFloat().unmask(NODATA)
file_name = GEE_AUX_IMAGE.split('.')[0]     # GEE appends the .tif suffix
task = ee.batch.Export.image.toDrive(
    image=conus_aux,
    description=file_name,
    folder=FOLDER,
    fileFormat='GeoTIFF',
    region=conus_bbox,
    scale=SCALE,
    crs=PROJ,
    skipEmptyTiles=True
)
task.start()

## Part 3: Add Climate Zone Data
Creates a new image from the GEE auxiliary data and the climate zone data. Reprojects the climate zone data to match the other auxiliary data.

Notes:
1. Run this once the export image GEE task has completed
2. Run the first two cells in the notebook before running this one if the kernel has been restarted since running part 2

In [10]:
import gdal
from gdal import gdalconst

In [11]:
gee_aux_file = os.path.join(GEE_DATA_DIR, GEE_AUX_IMAGE)
gee_aux_image = gdal.Open(gee_aux_file, gdal.GA_ReadOnly)
aux_bands = [gee_aux_image.GetRasterBand(b+1).GetDescription() for b in range(gee_aux_image.RasterCount)]
aux_bands.append('climate_zone')
print(aux_bands)
aux_data = gee_aux_image.ReadAsArray()
print(aux_data.shape)

['elevation', 'slope', 'aspect', 'longitude', 'latitude', 'climate_zone']
(5, 1448, 3341)


In [12]:
# Climate zone source
czone_file = os.path.join(common.SOURCE_DIR, KOPPEN_FILE)
czone_src = gdal.Open(czone_file, gdalconst.GA_ReadOnly)
czone_proj = czone_src.GetProjection()
czone_geotrans = czone_src.GetGeoTransform()

# Auxiliary projection and resolution
aux_proj = gee_aux_image.GetProjection()
aux_geotrans = gee_aux_image.GetGeoTransform()
x_size = gee_aux_image.RasterXSize
y_size = gee_aux_image.RasterYSize

# In-memory raster for the reprojected data
dst = gdal.GetDriverByName('MEM').Create("", x_size, y_size, 1, gdalconst.GDT_Byte)
dst.SetGeoTransform(aux_geotrans)
dst.SetProjection(aux_proj)

# Reproject climate zone data to auxiliary projection and resolution. Use the mode of the climate zones
gdal.ReprojectImage(czone_src, dst, czone_proj, aux_proj, gdalconst.GRA_Mode)

czone_data = dst.ReadAsArray()
czone_data = czone_data * (aux_data[0] != NODATA)
czone_data = np.where(czone_data==0, NODATA, czone_data)
czone_data.shape

(1448, 3341)

In [13]:
# Create output raster
aux_file = os.path.join(GEE_DATA_DIR, FINAL_AUX_IMAGE)
aux_driver = gdal.GetDriverByName('GTIFF')
aux_image = aux_driver.Create(aux_file, x_size, y_size, len(aux_bands), gdalconst.GDT_Float32, options=["TILED=YES"])
aux_image.SetGeoTransform(aux_geotrans)
aux_image.SetProjection(aux_proj)
for band in range(len(aux_bands)):
    print(f"Processing {aux_bands[band]} ...", end=' ')
    aux_band = aux_image.GetRasterBand(band + 1)
    aux_band.SetNoDataValue(NODATA)
    aux_band.SetDescription(aux_bands[band])
    if band == len(aux_bands) - 1:
        print("from czone data")
        aux_band.WriteArray(czone_data)
    else:
        print("from auxiliary data")
        aux_band.WriteArray(aux_data[band])
    aux_band.FlushCache()
del aux_image

Processing elevation ... from auxiliary data
Processing slope ... from auxiliary data
Processing aspect ... from auxiliary data
Processing longitude ... from auxiliary data
Processing latitude ... from auxiliary data
Processing climate_zone ... from czone data
